# NNMD of ammonia using the pretrained models

In this notebook, we perform a NN-based MD NVE simulation using ASE and the [SchNet NN potential](https://github.com/learningmatter-mit/NeuralForceField). We will be using the third generation of ammonia models, as shown in [our paper](https://arxiv.org/abs/2101.11588).

The utilities at `nff` will be used to perform the MD simulation. `nglview` will be used to visualize the generated trajectories.

In [1]:
import os
import numpy as np
from ase import Atoms, units
from ase.md.verlet import VelocityVerlet

from nff.io import NeuralFF, AtomsBatch, EnsembleNFF
from nff.md.nve import Dynamics
from nff.md.nvt import NoseHoover
from nff.data import Dataset
import nff.utils.constants as const

import nglview as nv
from ase.io import Trajectory, read

## Loading the models and parameters

The dataset contains a PyTorch dataset with all the DFT data used to train the third generation of NN potentials. Here, we will use the pre-trained ensemble. For learning how to train the models using the SchNet architecture and the current dataset, check the tutorials at the original [NFF repo](https://github.com/learningmatter-mit/NeuralForceField).

In [2]:
dset = Dataset.from_file('../data/ammonia.pth.tar')

`DEVICE` sets the GPU used for evaluating the model. If you want to evaluate on a CPU, set `DEVICE = 'cpu'`. The models are stored at `/models/ammonia`, where `/` is the root folder of this repository.

In [3]:
DEVICE = 2

def get_ensemble_calc(device=DEVICE):
    PATH = '../models/ammonia'
    models = []
    for model_name in sorted(os.listdir(PATH)):
        m = NeuralFF.from_file(os.path.join(PATH, model_name), device=device).model
        models.append(m)

    return EnsembleNFF(models, device=device)

ensemble = get_ensemble_calc(device=DEVICE)

Here, we set the parameters for the MD simulation. For learning how to use these parameters within our code, check the tutorials at the original [NFF repo](https://github.com/learningmatter-mit/NeuralForceField).

In [4]:
def get_md_params(traj_filename, temperature=1000):
    return {
        'T_init': temperature,
        'time_step': 1.0,
        'thermostat': VelocityVerlet,  
        'thermostat_params': {'timestep': 1.0 * units.fs},
        'steps': 5 * 1000,
        'save_frequency': 40,
        'nbr_list_update_freq': 10,
        'thermo_filename': 'thermo.log',
        'traj_filename': traj_filename,
        'skip': 0
    }

Finally, we use the lowest energy conformation within the existing dataset (i.e. the ground state of ammonia) as a starting configuration for the MD simulation. `AtomsBatch` is a [wrapper within our NFF repo](https://github.com/learningmatter-mit/NeuralForceField/blob/master/nff/io/ase.py) and can be used to interface an ASE atom with NFF.

In [5]:
CUTOFF = 5.0

def get_md_atoms(dset=dset, cutoff=CUTOFF, device=DEVICE):
    props = dset[np.argmin(dset.props['energy'])]

    atoms = AtomsBatch(
        positions=props['nxyz'][:, 1:],
        numbers=props['nxyz'][:, 0],
        cutoff=cutoff,
        props={'energy': 0, 'energy_grad': []},
        calculator=ensemble,
        device=device,
    )
    _ = atoms.update_nbr_list()
    
    return atoms

## Performing the MD simulation

Now, we perform the MD simulation using the parameters shown before.

In [6]:
TEMPERATURE = 1500

name = f'NVE_{TEMPERATURE}.traj'

atoms = get_md_atoms()
md_params = get_md_params(name, temperature=TEMPERATURE)
dyn = Dynamics(atoms, md_params)
traj = dyn.run()

/home/dskoda/.conda/envs/nff/lib/python3.7/site-packages/ase/md/md.py:48: FutureWarning: Specify the temperature in K using the 'temperature_K' argument
  warnings.warn(FutureWarning(w))
/home/dskoda/.conda/envs/nff/lib/python3.7/site-packages/ase/atoms.py:968: VisibleDeprecationWarning: Use get_global_number_of_atoms() instead
  np.VisibleDeprecationWarning)


Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           0.9087      -0.0017       0.9104  1760.8

0.0400           0.9214       0.2683       0.6532  1263.3

0.0800           0.9166       0.3652       0.5513  1066.3

0.1200           0.9147       0.3221       0.5925  1146.0

0.1600           0.9282       0.7402       0.1880   363.6

0.2000           0.9259       0.3814       0.5445  1053.1

0.2400           0.9245       0.4986       0.4260   823.8

0.2800           0.9210       0.2714       0.6496  1256.3

0.3200           0.9289       0.3034       0.6255  1209.7

0.3600           0.9235       0.2768       0.6467  1250.8

0.4000           0.9313       0.4563       0.4750   918.6

0.4400           0.9301       0.7316       0.1985   383.8

0.4800           0.9296       0.4780       0.4516   873.5

0.5200           0.9296       0.8318       0.0977   189.0

0.5600           0.9210       0.4233       0.4976   962.5

0.6000           0.9176       0.2918       0.6258  1210.3


## Visualizing the trajectory

Finally, we can visualize the trajectory on the Jupyter server using `nglview`. The filename was selected prior to the MD simulation.

In [7]:
filetraj = Trajectory(name)
view = nv.show_asetraj(filetraj)
view

NGLWidget(max_frame=125)